In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [ ]:
import inspect

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene, radiometry

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.terrain_correction))

In [ ]:
from dask.distributed import Client

# client = Client()
# client

# define input and load data

In [ ]:
product_urlpath = (
    "data/S1B_IW_SLC__1SDV_20211223T051121_20211223T051148_030148_039993_BA4B.SAFE/"
)
measurement_group = "IW3/VV"
dem_urlpath = "data/Rome-10m-DEM.tif"
orbit_group = None
calibration_group = None
output_urlpath = "Rome-10m-GTC-SLC.tif"
correct_radiometry = False
interp_method = "nearest"
multilook = None
grouping_area_factor = (1.0, 1.0)
open_dem_raster_kwargs = {"chunks": 1024}
chunks = 2048
correct_radiometry = "gamma_nearest"
kwargs = {}

In [ ]:
!ls -d {product_urlpath}
!ls -d {dem_urlpath}

In [ ]:
orbit_group = orbit_group or f"{measurement_group}/orbit"
calibration_group = calibration_group or f"{measurement_group}/calibration"

measurement = xr.open_dataarray(
    product_urlpath, engine="sentinel-1", group=measurement_group, chunks=chunks, **kwargs  # type: ignore
)

dem_raster = scene.open_dem_raster(dem_urlpath, **open_dem_raster_kwargs)

orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group, **kwargs)  # type: ignore
position_ecef = orbit_ecef.position
calibration = xr.open_dataset(product_urlpath, engine="sentinel-1", group=calibration_group, **kwargs)  # type: ignore
beta_nought_lut = calibration.betaNought

if measurement.attrs["product_type"] == "GRD":
    coordinate_conversion = xr.open_dataset(
        product_urlpath,
        engine="sentinel-1",
        group=f"{measurement_group}/coordinate_conversion",
        **kwargs,
    )  # type: ignore

position_ecef

# scene

In [ ]:
dem_raster

In [ ]:
_ = dem_raster.plot()

In [ ]:
%%time
dem_ecef = xr.map_blocks(scene.convert_to_dem_ecef, dem_raster)
dem_ecef = dem_ecef.drop_vars(dem_ecef.rio.grid_mapping)
dem_ecef

In [ ]:
# clean dask templates
template_raster = dem_raster.drop_vars(dem_raster.rio.grid_mapping)
template_3d = dem_ecef
template_3d

# acquisition

In [ ]:
measurement

In [ ]:
acquisition_template = xr.Dataset(
    data_vars={
        "azimuth_time": xr.full_like(template_raster, 0, dtype="datetime64[ns]"),
        "slant_range_time": template_raster,
        "dem_direction": template_3d,
    }
)

In [ ]:
%%time
acquisition = xr.map_blocks(
    apps.simulate_acquisition,
    dem_ecef,
    kwargs={"position_ecef": position_ecef},
    template=acquisition_template,
)
acquisition.dem_direction

In [ ]:
%%time
beta_nought = xarray_sentinel.calibrate_intensity(measurement, beta_nought_lut)
beta_nought

In [ ]:
%%time
if measurement.attrs["product_type"] == "GRD":
    ground_range = xarray_sentinel.slant_range_time_to_ground_range(
        acquisition.azimuth_time,
        acquisition.slant_range_time,
        coordinate_conversion,
    )
    interp_arg = ground_range
    interp_dim = "ground_range"
elif measurement.attrs["product_type"] == "SLC":
    interp_arg = acquisition.slant_range_time
    interp_dim = "slant_range_time"
    if measurement.attrs["mode"] == "IW":
        beta_nought = xarray_sentinel.mosaic_slc_iw(beta_nought)
else:
    raise ValueError(f"unsupported product_type {measurement.attrs['product_type']}")

interp_arg

In [ ]:
%%time
geocoded = apps.interpolate_measurement(
    acquisition.azimuth_time,
    interp_arg,
    beta_nought,
    multilook=multilook,
    interp_method=interp_method,
    interp_dim=interp_dim,
).chunk(dem_raster.chunksizes)

geocoded

In [ ]:
%%time
if correct_radiometry is not None:
    grid_parameters = radiometry.azimuth_slant_range_grid(
        measurement, coordinate_conversion, grouping_area_factor
    )

    if correct_radiometry == "gamma_bilinear":
        gamma_weights = radiometry.gamma_weights_bilinear
    elif correct_radiometry == "gamma_nearest":
        gamma_weights = radiometry.gamma_weights_nearest

    weights = xr.map_blocks(
        gamma_weights,
        dem_ecef,
        args=(acquisition,),
        kwargs=grid_parameters,
        template=template_raster,
    )
    geocoded = geocoded / weights

geocoded

In [ ]:
%%time
geocoded.load()

In [ ]:
%%time
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded.rio.to_raster(
    output_urlpath,
    dtype=np.float32,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)

In [ ]:
%%time
_ = geocoded.plot(vmax=0.4)